In [1]:
import ast
import pickle
import os
import pandas as pd

#### Luigi migration. Task yet to be created

In [25]:
import os
import sys
module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 


pars = load_parameters()

api_doc_file = pars.get('api_doc_file')
tagged_dataset_file = pars.get('tagged_dataset_file')
so_dump_processed_file = pars.get('so_dump_processed_file')
output_file = pars.get('output_file')

debug = pars.get('debug')

KeyError: 'PARS'

#### Utility to extract method name

In [2]:
from collections import deque

class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append(callvisitor.name)
    return func_calls

#### Load APIDoc and Dataset

In [6]:
api_doc_file = '../../data-import/build_api_doc_base/api_doc.csv'
tagged_dataset_file = '../../../data/stack-overflow/Dataset - Pandas.csv'
so_dump_processed_file = '../../../data/stack-overflow/pandas-preprocessedcode-dataset-part3'

api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)
tagged_dataset_df = pd.read_csv(tagged_dataset_file, encoding='ISO-8859-1', error_bad_lines=False)
processed_stackoverflow_df = pd.read_pickle(so_dump_processed_file)

def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        
        for index, row in api_df.iterrows():
            methodContext = row['SubCategory']
            #tokens = row['FullyQualifiedName'].split('.')
        
            #for token in tokens:
               #methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print('Error in method buildAPIDictionary',e)
    return api_dict
        

def buildTaggedDatasetDSForEvaluation(tagged_dataset_df):
    dataset_dict = dict()
    total_solutions = 0
    try:    
        for idx, row in tagged_dataset_df.iterrows():
            answerId = row['AnswerId']
            if answerId != 0:
                total_solutions = total_solutions +1
                tup = (int(row['SolutionId']), row['Solution'])
                if answerId in dataset_dict:
                    ls = dataset_dict[answerId]
                    ls.append(tup)
                    dataset_dict[answerId] = ls
                else:
                    ls = list()
                    ls.append(tup)
                    dataset_dict[answerId] = ls
    except Exception as e:
        print('Error in method buildTaggedDatasetDSForEvaluation',e)
        
    return dataset_dict, total_solutions

def buildStackOverflowDumpDict(processed_stackoverflow_df):
    stackoverflow_dict = dict()
    try:
        for idx, row in processed_stackoverflow_df.iterrows():
            postTypeId = row['PostTypeId']
            if postTypeId == 2:
                answerId = row['Id']
                stackoverflow_dict[answerId] = row['PreprocessedCode']
    except Exception as e:
        print('Error in method buildStackOverflowDumpDict', e)
    return stackoverflow_dict
    
api_dict = buildAPIDictionary(api_df)
tagged_dataset_dict, total_solutions = buildTaggedDatasetDSForEvaluation(tagged_dataset_df)
stackoverflow_dict = buildStackOverflowDumpDict(processed_stackoverflow_df)

Error in method buildTaggedDatasetDSForEvaluation cannot convert float NaN to integer


In [7]:
def lookUpAPIDocForContextH2(method_name):
    try:
        if method_name in api_dict.keys() and api_dict[method_name] == "Constructor":
            return False
        else:
            return True
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)
        
        
def lookUpAPIDocForContextH1H2(method_name):
    try:
        if method_name in api_dict.keys():
            if api_dict[method_name] == "Constructor":
                return False
            else:
                return True
        else:
            return True
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)

In [8]:
def applyH2():
    df_columns = ['ansId', 'line', 'actual','predicted']
    result_df = pd.DataFrame(columns=df_columns)

    df_row_id = 0


    for key in tagged_dataset_dict.keys():
        try:            
            solutionList = tagged_dataset_dict[key]
            content = str(stackoverflow_dict[key])
            lines = content.split(os.linesep)
            for line in lines:
                actualSolution = False
                predictedSolution = False
                for tup in solutionList:
                    if tup[1].strip() == line.strip():
                        actualSolution = True
                
                tree = ast.parse(line)
                
                func_calls = get_func_calls(tree)
                
                for func_call in func_calls:
                    tokens = func_call.split('.')
                    method_name = tokens[len(tokens)-1]
                    if lookUpAPIDocForContextH2(method_name):
                        predictedSolution = True
                        break
                result_df.loc[df_row_id] = [key,line, actualSolution, predictedSolution]
                df_row_id = df_row_id + 1
        except Exception as e:
            print('Error in method evaluate', e)
    return result_df
    

In [9]:
result_df = applyH2()
output_file = '../../../data/results/result_df_h2'
debug = True

result_df.to_pickle(output_file)

if debug:
    result_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)

Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected indent (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate un

In [10]:
def applyH1H2():
    df_columns = ['ansId', 'line', 'actual','predicted']
    result_df = pd.DataFrame(columns=df_columns)

    df_row_id = 0


    for key in tagged_dataset_dict.keys():
        try:            
            solutionList = tagged_dataset_dict[key]
            content = str(stackoverflow_dict[key])
            lines = content.split(os.linesep)
            for line in lines:
                actualSolution = False
                predictedSolution = False
                for tup in solutionList:
                    if tup[1].strip() == line.strip():
                        actualSolution = True
                
                tree = ast.parse(line)
                
                func_calls = get_func_calls(tree)
                
                for func_call in func_calls:
                    tokens = func_call.split('.')
                    method_name = tokens[len(tokens)-1]
                    if lookUpAPIDocForContextH1H2(method_name):
                        predictedSolution = True
                        break
                result_df.loc[df_row_id] = [key,line, actualSolution, predictedSolution]
                df_row_id = df_row_id + 1
        except Exception as e:
            print('Error in method evaluate', e)
    return result_df

In [11]:
result_df = applyH1H2()
output_file = '../../../data/results/result_df_h1h2'
debug = True

result_df.to_pickle(output_file)

if debug:
    result_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)


Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected indent (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate un